In [10]:
import pandas as pd
import numpy as np
import re
import datetime
from sklearn.metrics import median_absolute_error

# Read preprocessed csv data

In [11]:
data_features = pd.read_csv('data/preprocessed.csv')
data_features[0:10]

,host_neighbourhood,host_identity_verified,neighbourhood,city,zipcode,property_type,room_type,accommodates,bathrooms,bedrooms,...,availability_365,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,cancellation_policy
0,Harlem,t,Harlem,New York,10026,Apartment,Private room,2,1.000000,1.0,...,316,164,92.000000,9.000000,9.000000,10.00000,9.000000,9.0000,9.000000,strict
1,Gravesend,t,Kensington,Brooklyn,11218,Apartment,Private room,4,1.000000,2.0,...,365,7,97.000000,10.000000,10.000000,10.00000,10.000000,10.0000,10.000000,moderate
2,Midtown,t,Midtown,New York,10018,Apartment,Entire home/apt,2,1.000000,0.0,...,365,29,97.000000,9.000000,9.000000,9.00000,10.000000,10.0000,9.000000,strict
3,East Williamsburg,t,Williamsburg,Brooklyn,11206,Apartment,Private room,2,1.000000,1.0,...,279,27,97.000000,10.000000,10.000000,10.00000,10.000000,10.0000,10.000000,strict
4,Harlem,t,Harlem,New York,10027,Apartment,Private room,2,1.000000,1.0,...,365,0,93.559763,9.575481,9.242865,9.72651,9.744402,9.4733,9.395225,strict
5,Clinton Hill,t,Clinton Hill,Brooklyn,11238,Other,Entire home/apt,3,1.000000,1.0,...,126,181,90.000000,9.000000,9.000000,10.00000,10.000000,9.0000,9.000000,moderate
6,Hell's Kitchen,t,Hell's Kitchen,New York,10036,Apartment,Entire home/apt,3,1.000000,1.0,...,17,26,88.000000,9.000000,9.000000,10.00000,10.000000,10.0000,9.000000,strict
7,Times Square,f,Hell's Kitchen,New York,10019,Apartment,Entire home/apt,2,1.000000,1.0,...,2,58,93.000000,9.000000,8.000000,10.00000,10.000000,10.0000,9.000000,strict
8,Bedford-Stuyvesant,f,Bedford-Stuyvesant,Brooklyn,11216,Apartment,Private room,2,1.132942,1.0,...,365,51,90.000000,8.000000,8.000000,10.00000,10.000000,9.0000,9.000000,strict
9,Greenwood Heights,f,Greenwood Heights,Brooklyn,11232,Apartment,Entire home/apt,4,1.500000,3.0,...,358,1,97.000000,10.000000,10.000000,10.00000,10.000000,8.0000,10.000000,strict


# Read Class Labels (Prices)

In [12]:
data_values = pd.read_csv('data/airbnb_class.csv')
prices = []
def removeDollar(col):
    for i in range(len(data_values[col])):
        price = data_values['price'][i]
        price = re.sub('[,$]', '', price)
        prices.append(float(price))
    print prices[0:3] , len(prices)

# Converting Price values to float [Class Labels] 

In [13]:
print data_values[0:3]
removeDollar('price')

     price
0   $59.00
1   $39.00
2  $225.00
[59.0, 39.0, 225.0] 48852


In [14]:
from collections import defaultdict
dd = defaultdict(list)
dict_data_features = data_features.to_dict('records')

In [15]:
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer()
data_features_array = vec.fit_transform(dict_data_features).toarray()
print data_features_array.shape
data_features_array

(48852L, 1403L)


array([[  2.,   9., 316., ...,   0.,   0.,   0.],
       [  4.,  30., 365., ...,   0.,   0.,   0.],
       [  2.,  30., 365., ...,   0.,   0.,   0.],
       ...,
       [  3.,   0.,   0., ...,   0.,   0.,   0.],
       [  4.,   1.,   1., ...,   0.,   0.,   0.],
       [  1.,   2.,  30., ...,   0.,   0.,   0.]])

# splitting the data set to train & test

In [16]:
from sklearn.model_selection import train_test_split
start = datetime.datetime.now()
X_train, X_test, y_train, y_test = train_test_split(data_features_array, prices, test_size=0.2, random_state=1)
print X_train.shape, X_test.shape

(39081L, 1403L) (9771L, 1403L)


In [17]:
from sklearn import linear_model
reg = linear_model.LinearRegression()
start = datetime.datetime.now()
reg.fit(X_train, y_train)
end = datetime.datetime.now()
print  "Time taken: ", end - start
predict = reg.predict(X_test)
print "MAE: ", median_absolute_error(y_test, predict)

Time taken:  0:00:26.285000
MAE:  36.73052978515625


In [26]:
from sklearn.linear_model import SGDClassifier
start = datetime.datetime.now()
clf = SGDClassifier(loss="log", penalty="l1")
clf.fit(X_train, y_train)
end = datetime.datetime.now()
print "Time taken: ", end - start
predict1 = clf.predict(X_test)
print "MAE: ", median_absolute_error(y_test, predict1)

Time taken:  0:14:53.632000
MAE:  45.0


In [19]:
from sklearn import linear_model
start = datetime.datetime.now()
reg = linear_model.Ridge(alpha = 1000.0)
reg.fit (X_train, y_train) 
end = datetime.datetime.now()
print  "Time taken: ", end - start
predict1 = reg.predict(X_test)
print median_absolute_error(y_test, predict1)

Time taken:  0:00:11.968000
33.35164361856056


In [20]:
from sklearn import linear_model
start = datetime.datetime.now()
reg = linear_model.LassoLars(alpha=0.06)
reg.fit (X_train, y_train) 
end = datetime.datetime.now()
print  "Time taken: ", end - start
predict1 = reg.predict(X_test)
print "MAE: ", median_absolute_error(y_test, predict1)

Time taken:  0:00:05.212000
MAE:  33.80188430181772


In [21]:
start = datetime.datetime.now()
reg = linear_model.BayesianRidge(alpha_1=0.01)
reg.fit (X_train, y_train) 
end = datetime.datetime.now()
print  "Time taken: ", end - start
predict1 = reg.predict(X_test)
print "MAE: ", median_absolute_error(y_test, predict1)

Time taken:  0:00:35.665000
MAE:  35.039555470611646


In [27]:
prices = np.asarray(prices)
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10)
lass = []
rid = []
avg = []
for train, test in skf.split(data_features_array, prices):
    X_train = data_features_array[train]
    X_test = data_features_array[test]
    y_train = prices[train]
    y_test = prices[test]
    start = datetime.datetime.now()
    reg = linear_model.LassoLars(alpha=0.06)
    reg.fit (X_train, y_train) 
    end = datetime.datetime.now()
    print  "Time taken for lasso lars: ", end - start
    predict1 = reg.predict(X_test)
    lassoLars = median_absolute_error(y_test, predict1)
    lass.append(lassoLars)
    print "MAE for Lasso Lars: ", lassoLars
    start = datetime.datetime.now()
    reg = linear_model.Ridge(alpha = 1000.0)
    reg.fit (X_train, y_train) 
    end = datetime.datetime.now()
    print  "Time taken for ridge regression: ", end - start
    predict2 = reg.predict(X_test)
    ridge = median_absolute_error(y_test, predict2)
    rid.append(ridge)
    print "MAE for ridge regression: ", median_absolute_error(y_test, predict2)
    print "Difference: ", lassoLars - ridge
    print "Average MAE: ", median_absolute_error(y_test, ((predict1 + predict2)/2.0))
    avg.append(median_absolute_error(y_test, ((predict1 + predict2)/2.0)))
total = 0.00
for i in lass:
    total = total + float(i)
print total/len(lass)

total = 0.00
for i in rid:
    total = total + float(i)
print total/len(rid)

total = 0.00
for i in avg:
    total = total + float(i)
print total/len(avg)

Time taken for lasso lars:  0:00:20.286000
MAE for Lasso Lars:  36.163107544196976
Time taken for ridge regression:  0:00:05.979000
MAE for ridge regression:  33.126900498248176
Difference:  3.0362070459487995
Average MAE:  30.864148110142267
Time taken for lasso lars:  0:00:03.705000
MAE for Lasso Lars:  34.80869113728913
Time taken for ridge regression:  0:00:03.055000
MAE for ridge regression:  32.49685101581886
Difference:  2.3118401214702686
Average MAE:  29.789418038997965
Time taken for lasso lars:  0:00:10.180000
MAE for Lasso Lars:  35.36299177202366
Time taken for ridge regression:  0:00:12.176000
MAE for ridge regression:  33.15665187258989
Difference:  2.2063398994337717
Average MAE:  31.769743802806286
Time taken for lasso lars:  0:00:09.152000
MAE for Lasso Lars:  34.91894424842488
Time taken for ridge regression:  0:00:05.532000
MAE for ridge regression:  31.86759352096331
Difference:  3.0513507274615677
Average MAE:  30.367463542729134
Time taken for lasso lars:  0:00:0

In [23]:
total = 0.00
for i in avg:
    total = total + float(i)
print total/len(avg)

31.5743803153


In [24]:
predictValues = [{
  'accommodates': 2,
  'availability_30': 24,
  'availability_365': 170,
  'availability_60': 54,
  'availability_90': 80,
  'bathrooms': 1.0,
  'bed_type': 'Real Bed',
  'bedrooms': 1.0,
  'beds': 1.0,
  'cancellation_policy': 'moderate',
  'city': 'Bronx',
  'cleaning_fee': '$50.00',
  'extra_people': '$20.00',
  'guests_included': 1.0,
  'host_identity_verified': 't',
  'host_neighbourhood': 'City Island',
  'neighbourhood': 'City Island',
  'number_of_reviews': 25,
  'property_type': 'House',
  'review_scores_accuracy': 10.0,
  'review_scores_checkin': 10.0,
  'review_scores_cleanliness': 10.0,
  'review_scores_communication': 10.0,
  'review_scores_location': 10.0,
  'review_scores_rating': 100.0,
  'review_scores_value': 10.0,
  'room_type': 'Private room',
  'security_deposit': '$100.00',
  'square_feet': 718.1781305110001,
  'zipcode': '10464'
}]

In [25]:
hello = vec.transform(predictValues)
print "hello, array: ", hello.toarray()
reg = linear_model.Ridge(alpha = 1000.0)
reg.fit (X_train, y_train) 
print "reg: ", reg
super_predict = reg.predict(hello.toarray())
print "Predicted value is:-->", super_predict
price = str(super_predict[0])
print price

hello, array:  [[  2.  24. 170. ...   0.   0.   0.]]
reg:  Ridge(alpha=1000.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)
Predicted value is:--> [57.24562104]
57.24562104078424
